# GSDS FAQ RAG 챗봇 실습

**실습 목표**
- RAG(Retrieval-Augmented Generation)의 개념과 필요성 이해
- 임베딩(Embedding)과 벡터 유사도 체험
- 실제 웹사이트에서 데이터 수집 및 전처리
- 벡터 데이터베이스 구축 및 검색
- 근거 기반 FAQ 챗봇 구현

---

## RAG(Retrieval-Augmented Generation)란?

### 문제 상황

일반 LLM에게 질문하면:
- **환각(Hallucination)**: 없는 정보를 그럴듯하게 지어냄
- **정보 부족**: 학습 데이터에 없는 최신 정보나 특정 도메인 지식 부족
- **출처 불명**: 어디서 나온 정보인지 알 수 없음

**예시:**
```
Q: "서울대 데이터사이언스대학원 석사 졸업 학점은?"
LLM (No RAG): "일반적으로 30학점 정도입니다" ← 추측
```

### RAG의 해결책

**RAG = Retrieval (검색) + Augmented (강화된) + Generation (생성)**

```
┌─────────────┐
│  사용자 질문   │
└──────┬──────┘
       ↓
┌──────────────────┐
│ 1) 벡터 DB 검색    │  ← 관련 문서 찾기
└──────┬───────────┘
       ↓
┌──────────────────┐
│ 2) 검색된 문서 +    │  ← 근거 자료 제공
│    질문           │
└──────┬───────────┘
       ↓
┌──────────────────┐
│ 3) 근거 기반 답변   │  ← 출처 있는 답변
└──────────────────┘
```

### RAG의 장점

- **정확성**: 실제 문서에 기반한 답변
- **출처 제공**: 어떤 문서에서 나온 정보인지 명시
- **최신 정보**: 벡터 DB만 업데이트하면 최신 정보 반영
- **도메인 특화**: 특정 기관/회사 문서로 전문 챗봇 구축

### 오늘 실습

**GSDS(서울대 데이터사이언스대학원) FAQ 챗봇**
- 입학/졸업 요건
- 수강신청 절차
- 휴학/복학 안내
- 기타 학사 정보

→ GSDS 웹사이트에서 실제 데이터를 수집하여 구축

---

## 0. 환경 설정

### 필요한 라이브러리 설치

In [ ]:
# 필요한 라이브러리 설치
!pip install -q openai chromadb beautifulsoup4 trafilatura tiktoken tqdm numpy requests

### OpenAI API 키 설정

In [2]:
import os

OPENAI_API_KEY = "sk-YOUR_API_KEY"  # FIXME

os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

---

## Part A: 임베딩 이해

### 임베딩(Embedding)이란?

**임베딩**: 텍스트(단어, 문장, 문서)를 숫자 벡터로 변환하는 기술

```
"졸업 요건이 궁금해요"  →  [0.2, -0.5, 0.8, ..., 0.1]  (1536차원 벡터)
"수료 학점은 몇 점인가요"  →  [0.3, -0.4, 0.7, ..., 0.2]  (비슷한 벡터)
"데이터베이스 과제 도와줘" →  [-0.6, 0.1, -0.3, ..., 0.9]  (다른 벡터)
```

**핵심 아이디어:**
- 의미가 비슷한 텍스트 → 벡터가 가까움 (유사도 높음)
- 의미가 다른 텍스트 → 벡터가 멂 (유사도 낮음)

### 문장 유사도 측정

In [3]:
import numpy as np
from openai import OpenAI

client = OpenAI()
EMBED_MODEL = "text-embedding-3-small"

def embed_texts(texts, model=EMBED_MODEL):
    """
    text list를 임베딩 벡터로 변환합니다.

    Args:
        texts: text list
        model: OpenAI embedding model

    Returns:
        numpy array of embeddings (shape: [n_texts, embedding_dim])
    """
    resp = client.embeddings.create(model=model, input=texts)
    return np.array([d.embedding for d in resp.data], dtype=np.float32)

def cosine_similarity(a, b):
    """
    두 벡터 간의 코사인 유사도를 계산합니다.

    코사인 유사도:
    - 1.0에 가까울수록 매우 유사
    - 0.0에 가까울수록 관련 없음
    - -1.0에 가까울수록 반대 의미
    """
    # 벡터 정규화
    a = a / (np.linalg.norm(a) + 1e-9)
    b = b / (np.linalg.norm(b) + 1e-9)
    # 내적 계산
    return float(np.dot(a, b))


### 예제: 졸업 관련 질문 vs 다양한 문장들

In [4]:
# 샘플 문장들
sample_sentences = [
    "졸업 요건이 궁금해요.",
    "수료 학점이 몇 학점인가요?",
    "휴학 신청 절차를 알려주세요.",
    "데이터베이스 수업 과제 도와줘.",
    "오늘 날씨가 정말 좋네요.",
]

# 질문
question = "졸업하려면 몇 학점이 필요해요?"

# 모든 문장 임베딩
print("임베딩 생성 중...\n")
all_texts = [question] + sample_sentences
embeddings = embed_texts(all_texts)

question_emb = embeddings[0]
sentence_embs = embeddings[1:]

print(f"임베딩 완료!")
print(f"   - 벡터 차원: {len(question_emb)}차원")
print(f"   - 벡터 샘플 (앞 10개): {question_emb[:10]}\n")

# 유사도 계산
print("="*70)
print(f"질문: \"{question}\"")
print("="*70)
print("\n각 문장과의 유사도 (높을수록 의미가 비슷함):\n")

similarities = []
for i, sent in enumerate(sample_sentences):
    sim = cosine_similarity(question_emb, sentence_embs[i])
    similarities.append((sent, sim))

# 유사도 순으로 정렬
similarities_sorted = sorted(similarities, key=lambda x: x[1], reverse=True)

for i, (sent, sim) in enumerate(similarities_sorted, 1):
    bar = "█" * int(sim * 20)
    print(f"{i}. [{sim:.3f}] {bar}")
    print(f"   {sent}\n")

임베딩 생성 중...

임베딩 완료!
   - 벡터 차원: 1536차원
   - 벡터 샘플 (앞 10개): [-0.01340493  0.04384678 -0.00534345  0.00823172  0.01111104  0.02494607
 -0.03847047 -0.02019102 -0.00091995  0.00402626]

질문: "졸업하려면 몇 학점이 필요해요?"

각 문장과의 유사도 (높을수록 의미가 비슷함):

1. [0.578] ███████████
   수료 학점이 몇 학점인가요?

2. [0.568] ███████████
   졸업 요건이 궁금해요.

3. [0.342] ██████
   휴학 신청 절차를 알려주세요.

4. [0.215] ████
   데이터베이스 수업 과제 도와줘.

5. [0.015] 
   오늘 날씨가 정말 좋네요.



## Part B: 데이터 수집 & 청킹

### B-1. GSDS 웹사이트에서 문서 수집

실제 GSDS 웹사이트에서 핵심 페이지들을 수집합니다.

**수집 대상:**
- 모집요강
- 입학/졸업 요건
- FAQ
- 수강신청 안내
- 휴학/복학 안내

In [5]:
# 수집할 URL 리스트
SEED_URLS = [
    # 모집/입학/FAQ
    "https://gsds.snu.ac.kr/academics/recruitment/",
    "https://gsds.snu.ac.kr/academics/acadmic-admission/",
    "https://gsds.snu.ac.kr/gsds-faq/",

    # 이수/졸업
    "https://gsds.snu.ac.kr/academics/complete/",
    "https://gsds.snu.ac.kr/academics/qual/",

    # 수강신청 (학기 공지 예시)
    "https://gsds.snu.ac.kr/2025%ED%95%99%EB%85%84%EB%8F%84-2%ED%95%99%EA%B8%B0-%EC%88%98%EA%B0%95%EC%8B%A0%EC%B2%AD-%EC%95%88%EB%82%B4/",
    "https://gsds.snu.ac.kr/2025%ED%95%99%EB%85%84%EB%8F%84-2%ED%95%99%EA%B8%B0-%EC%88%98%EA%B0%95%EC%8B%A0%EC%B2%AD-%EB%B3%80%EA%B2%BD-%EB%B0%8F-%EC%B7%A8%EC%86%8C-%EC%9D%BC%EC%A0%95-%EC%95%88%EB%82%B4/",

    # 휴학/복학 (학기 공지 예시)
    "https://gsds.snu.ac.kr/2024%ED%95%99%EB%85%84%EB%8F%84-2%ED%95%99%EA%B8%B0-%ED%9C%B4%ED%95%99-%EB%B3%B5%ED%95%99%EA%B7%80-%EB%B3%B5%EC%A0%81%EC%9E%AC%EC%9E%85%ED%95%99-%EB%B0%8F-%EB%93%B1%EB%A1%9D-%EC%9D%BC/",
]

print(f"수집 url: {len(SEED_URLS)}개 페이지")
for i, url in enumerate(SEED_URLS, 1):
    from urllib.parse import unquote
    readable = unquote(url.split("/")[-2] if url.endswith("/") else url.split("/")[-1])
    print(f"{i:2d}. {readable[:50]}...")

수집 url: 8개 페이지
 1. recruitment...
 2. acadmic-admission...
 3. gsds-faq...
 4. complete...
 5. qual...
 6. 2025학년도-2학기-수강신청-안내...
 7. 2025학년도-2학기-수강신청-변경-및-취소-일정-안내...
 8. 2024학년도-2학기-휴학-복학귀-복적재입학-및-등록-일...


### HTML에서 본문 추출하기

웹페이지는 메뉴, footer, siderbar 등 불필요한 내용이 많습니다.

**사용 도구:**
- `trafilatura`: 본문 자동 추출 (메인 도구)
- `BeautifulSoup`: trafilatura 실패 시 대체

In [6]:
import requests
from bs4 import BeautifulSoup
import trafilatura
from tqdm import tqdm

# User-Agent 설정
HEADERS = {
    "User-Agent": "Mozilla/5.0 (compatible; GSDS-RAG-Lab/1.0; +https://gsds.snu.ac.kr/)"
}

def fetch_html(url: str, timeout: int = 20) -> str:
    """URL에서 HTML 가져오기"""
    r = requests.get(url, headers=HEADERS, timeout=timeout)
    r.raise_for_status()
    return r.text

def extract_title(html: str) -> str:
    """HTML에서 제목 추출"""
    soup = BeautifulSoup(html, "html.parser")

    # <title> 태그 우선
    if soup.title and soup.title.get_text(strip=True):
        return soup.title.get_text(strip=True)

    # <h1> 태그 대체
    h1 = soup.find("h1")
    return h1.get_text(strip=True) if h1 else "Untitled"

def extract_text(html: str) -> str:
    """
    HTML에서 본문 텍스트 추출

    1차: trafilatura (메인 컨텐츠 자동 추출)
    2차: BeautifulSoup (fallback)
    """
    # trafilatura로 시도
    extracted = trafilatura.extract(
        html,
        include_comments=False,
        include_tables=True
    )

    if extracted and len(extracted.strip()) > 200:
        return extracted.strip()

    # BeautifulSoup으로 대체
    soup = BeautifulSoup(html, "html.parser")

    # 불필요한 태그 제거
    for tag in soup(["script", "style", "noscript"]):
        tag.decompose()

    # 텍스트 추출
    text = soup.get_text("\n")
    lines = [ln.strip() for ln in text.splitlines() if ln.strip()]
    return "\n".join(lines)


### 실제 데이터 수집 실행

In [8]:
print("데이터 수집 시작...\n")

documents = []

for url in tqdm(SEED_URLS, desc="페이지 수집"):
    try:
        # HTML 가져오기
        html = fetch_html(url)

        # 제목과 본문 추출
        title = extract_title(html)
        text = extract_text(html)

        documents.append({
            "url": url,
            "title": title,
            "text": text
        })

    except Exception as e:
        print(f"\n수집 실패: {url}")
        print(f"   에러: {repr(e)}")

print(f"\n수집 완료: {len(documents)}개 페이지\n")

# 샘플 확인
if documents:
    sample = documents[0]
    print("="*70)
    print("샘플 문서")
    print("="*70)
    print(f"제목: {sample['title']}")
    print(f"URL: {sample['url']}")
    print(f"본문 길이: {len(sample['text'])}자")
    print(f"\n본문 미리보기 (앞 500자):\n")
    print(sample['text'][:500])
    print("\n...")

데이터 수집 시작...



페이지 수집: 100%|██████████| 8/8 [00:06<00:00,  1.24it/s]


수집 완료: 8개 페이지

샘플 문서
제목: 모집안내 – 서울대학교 데이터사이언스대학원
URL: https://gsds.snu.ac.kr/academics/recruitment/
본문 길이: 487자

본문 미리보기 (앞 500자):

2026학년도 전기 모집 안내
우수 인재의 조기 확보를 위하여 2026학년도 신입생 선발 시 전기모집에서 전·후기 정원을 통합하여 신입생을 선발함. 전기모집에서 전·후기 전체 모집인원을 충원할 경우, 후기모집은 진행하지 않을 수 있음(미실시 여부는 2026학년도 전기 합격자 최종 등록 마감 후 데이터사이언스대학원 홈페이지에 공고 예정임)
| 학위과정 | 대학원 | 모집단위 | 모집인원 |
|---|---|---|---|
| 전문석사 | 데이터사이언스대학원 | 데이터사이언스학과 | 18명 |
| 석사‧박사통합 | 데이터사이언스대학원 | 데이터사이언스학과 | 60명 |
| 박사과정 | 데이터사이언스대학원 | 데이터사이언스학과 | 25명 |
○ 석사‧박사통합과정: 학사학위자가 석사학위 및 박사학위의 과정이 통합된 과정을 이수하는 것으로서, 학생이 통합과정 이수중단을 원할 시 학칙 및 규정이 정하는 기준에 따라 석사학위가 수여됨
더 자세한 사항은 입학공지 게시판 참조

...


### B-2. 청킹(Chunking): 문서를 작은 조각으로 나누기

**왜 청킹이 필요한가?**

1. **검색 정확도**: 문서 전체보다 관련 부분만 찾는 것이 더 정확
2. **LLM 컨텍스트**: LLM에 너무 긴 텍스트를 넣으면 중요한 정보를 놓칠 수 있음
3. **비용**: 작은 청크만 사용하면 토큰 비용 절감

**청킹 전략:**
- `chunk_size`: 청크 하나의 크기 (토큰 기준)
- `overlap`: 청크 간 중복 영역 (문맥 유지)

```
[   Chunk 1   ]
            [overlap]
               [   Chunk 2   ]
                           [overlap]
                              [   Chunk 3   ]
```

In [9]:
import tiktoken

# OpenAI 토크나이저 초기화
tokenizer = tiktoken.get_encoding("cl100k_base")

def chunk_text(text: str, chunk_size: int = 350, overlap: int = 50):
    """
    텍스트를 토큰 기반으로 청킹합니다.

    Args:
        text: 원본 텍스트
        chunk_size: 청크당 토큰 수
        overlap: 청크 간 중복 토큰 수

    Returns:
        청크 리스트
    """
    # 텍스트를 토큰으로 변환
    tokens = tokenizer.encode(text)
    chunks = []
    start = 0

    while start < len(tokens):
        # 청크 끝 위치
        end = min(start + chunk_size, len(tokens))

        # 토큰을 다시 텍스트로 변환
        chunk = tokenizer.decode(tokens[start:end])
        chunks.append(chunk)

        # 마지막 청크면 종료
        if end == len(tokens):
            break

        # 다음 시작 위치 (overlap 적용)
        start = end - overlap

    return chunks


### 모든 문서를 청킹

In [10]:
# 청킹 파라미터
CHUNK_SIZE = 350   # 청크당 토큰 수
OVERLAP = 50       # 중복 토큰 수

print(f"청킹 시작 (chunk_size={CHUNK_SIZE}, overlap={OVERLAP})...\n")

chunks = []

for doc in documents:
    # 문서를 청크로 분할
    doc_chunks = chunk_text(doc["text"], chunk_size=CHUNK_SIZE, overlap=OVERLAP)

    # 각 청크에 메타데이터 추가
    for i, chunk in enumerate(doc_chunks):  # chunk_text → chunk로 변경
        chunks.append({
            "id": f"{abs(hash(doc['url']))}_{i}",  # 고유 ID
            "text": chunk,  # chunk_text → chunk로 변경
            "meta": {
                "url": doc["url"],
                "title": doc["title"],
                "chunk_id": i  # 문서 내 청크 번호
            }
        })

print(f"chunking 완료!\n")
print(f"통계:")
print(f"   - 원본 문서: {len(documents)}개")
print(f"   - 총 청크 수: {len(chunks)}개")
print(f"   - 문서당 평균 청크 수: {len(chunks)/max(len(documents),1):.1f}개\n")

# 샘플 청크 확인
if chunks:
    sample_chunk = chunks[0]
    print("="*70)
    print("샘플 청크")
    print("="*70)
    print(f"ID: {sample_chunk['id']}")
    print(f"메타데이터: {sample_chunk['meta']}")
    print(f"\n내용 (앞 300자):\n")
    print(sample_chunk['text'][:300])
    print("\n...")

청킹 시작 (chunk_size=350, overlap=50)...

chunking 완료!

통계:
   - 원본 문서: 8개
   - 총 청크 수: 91개
   - 문서당 평균 청크 수: 11.4개

샘플 청크
ID: 3074608195250215714_0
메타데이터: {'url': 'https://gsds.snu.ac.kr/academics/recruitment/', 'title': '모집안내 – 서울대학교 데이터사이언스대학원', 'chunk_id': 0}

내용 (앞 300자):

2026학년도 전기 모집 안내
우수 인재의 조기 확보를 위하여 2026학년도 신입생 선발 시 전기모집에서 전·후기 정원을 통합하여 신입생을 선발함. 전기모집에서 전·후기 전체 모집인원을 충원할 경우, 후기모집은 진행하지 않을 수 있음(미실시 여부는 2026학년도 전기 합격자 최종 등록 마감 후 데이터사이언스대학원 홈페이지에 공고 예정임)
| 학위과정 | 대학원 | 모집단위 | 모집인원 |
|---|---|---|---|
| 전문석사 | 데이터사이언스대학원 | 데이터사이언스학과 | 18명 |
| 석사‧박사통합 | 데이터사이언스대학원 

...


## Part C: 벡터 DB 구축

### VectorDB란?
**VectorDB**는 텍스트/이미지 같은 데이터를 임베딩 벡터 (숫자 리스트)로 저장해두고, 질의 벡터와 가까운 벡터를 빠르게 찾는 (유사도 검색) 데이터베이스

**필요 이유**
- 키워드 검색은 "단어가 정확히 포함되었는지"만 보지만 vectordb는 "의미가 비슷한지"를 보고 찾을수 있음.

**동작 방식**
- 문서청크 → 임베딩(벡터화) → 벡터디비에 저장
- 질문 → 임베딩(벡터화) → 벡터디비에서 top-k 유사 청크 검색
- 실무에서는 빠른 검색을 위해 ANN 인덱스(HNSW등)을 사용함.
- 벡터만 저장하는게 아니라, 보통 (원문 텍스트 + metadata (url, 제목, chunk_id 등) + 벡터)를 함께 저장해서 "찾은 뒤 원문을 가져오는" 방식으로 사용함.

**유사도는 어떻게 재는가**
- 보통 cosine similarity (방향이 비슷한지) 혹은 L2 distance (거리)


<br/>

### ChromaDB란?

**ChromaDB**: 벡터데이터베이스 중 한 종류, 파이썬에서 로컬로 쉽게 쓰기 좋음.


**주요 기능:**
- 벡터 저장 및 인덱싱
- 유사도 기반 검색 (cosine similarity)
- 메타데이터 필터링
- 로컬 파일 저장 (PersistentClient)

### ChromaDB 초기화

In [11]:
import chromadb

# chromadb에서 collection: 벡터들을 모아두는 논리적 컨테이너 (테이블 단위라고 보면 좋음)

# 저장 경로 및 컬렉션 이름
CHROMA_PATH = "./chroma_gsds"           # 벡터 DB 저장 경로
COLLECTION_NAME = "gsds_faq_rag"        # 컬렉션 이름

# ChromaDB 클라이언트 초기화
# client: chromadb에 연결해서 컬렉션을 만들고/저장하고/검색하는 핸들러
chroma_client = chromadb.PersistentClient(path=CHROMA_PATH)

# 기존 컬렉션 삭제 (재실행 시 중복 방지)
try:
    chroma_client.delete_collection(COLLECTION_NAME)
    print("기존 컬렉션 삭제")
except Exception:
    pass

# 새 컬렉션 생성
collection = chroma_client.get_or_create_collection(
    name=COLLECTION_NAME,
    metadata={"hnsw:space": "cosine"}  # 코사인 유사도 사용
)

print(f"ChromaDB 초기화 완료")
print(f"   - 저장 경로: {CHROMA_PATH}")
print(f"   - 컬렉션: {COLLECTION_NAME}")

ChromaDB 초기화 완료
   - 저장 경로: ./chroma_gsds
   - 컬렉션: gsds_faq_rag


### 청크 임베딩 및 저장

모든 청크를 임베딩하고 ChromaDB에 저장합니다.


In [12]:
BATCH_SIZE = 64  # 한 번에 처리할 청크 수

print(f"청크 임베딩 및 저장 시작 (배치 크기: {BATCH_SIZE})...\n")

for i in tqdm(range(0, len(chunks), BATCH_SIZE), desc="배치 처리"):
    # 배치 추출
    batch = chunks[i:i+BATCH_SIZE]

    # 배치 데이터 준비
    ids = [c["id"] for c in batch]
    texts = [c["text"] for c in batch]
    metas = [c["meta"] for c in batch]

    # 임베딩 생성
    embeddings = embed_texts(texts)

    # ChromaDB에 저장
    collection.add(
        ids=ids,
        embeddings=embeddings.tolist(),
        documents=texts,
        metadatas=metas
    )

print(f"\n✅ 저장 완료!")
print(f"   - 컬렉션 내 문서 수: {collection.count()}개")

청크 임베딩 및 저장 시작 (배치 크기: 64)...



배치 처리: 100%|██████████| 2/2 [00:02<00:00,  1.14s/it]


✅ 저장 완료!
   - 컬렉션 내 문서 수: 91개


### 벡터 검색 테스트

In [13]:
def search_documents(question: str, k: int = 3):
    """
    질문과 관련된 문서 청크를 검색합니다.

    Args:
        question: 검색 질문
        k: 반환할 결과 수

    Returns:
        관련 문서 리스트
    """
    # 질문 임베딩
    q_emb = embed_texts([question])[0]

    # ChromaDB 검색
    results = collection.query(
        query_embeddings=[q_emb.tolist()],
        n_results=k,
        include=["documents", "metadatas", "distances"]
    )

    # 결과 포맷팅
    hits = []
    for doc, meta, dist in zip(
        results["documents"][0],
        results["metadatas"][0],
        results["distances"][0]
    ):
        hits.append({
            "text": doc,
            "meta": meta,
            "distance": dist,  # 거리 (낮을수록 유사)
            "similarity": 1 - dist  # 유사도 (높을수록 유사)
        })

    return hits

# 테스트 질문
test_question = "석사 졸업에 필요한 학점은 몇 학점인가요?"

print("="*70)
print(f"검색 테스트")
print("="*70)
print(f"질문: {test_question}\n")

results = search_documents(test_question, k=3)

for i, hit in enumerate(results, 1):
    print(f"\n결과 #{i}")
    print(f"   유사도: {hit['similarity']:.3f}")
    print(f"   제목: {hit['meta']['title']}")
    print(f"   URL: {hit['meta']['url']}")
    print(f"   내용 (앞 200자): {hit['text'][:200]}...")

검색 테스트
질문: 석사 졸업에 필요한 학점은 몇 학점인가요?


결과 #1
   유사도: 0.586
   제목: 이수규정 – 서울대학교 데이터사이언스대학원
   URL: https://gsds.snu.ac.kr/academics/complete/
   내용 (앞 200자): 을 이수하여야 하며, 최대 12학점까지 수료학점에 포함된다.
다. 본 대학원 석사과정에서 수료 기준학점을 초과하여 취득한 학점(기초 분야 제외)에 대하여 12학점까지 박사과정 수료학점으로 인정할 수 있다. 단, 석사과정에서 이수한 교과목을 박사과정에서 다시 이수하였을 경우에는 중복이수가 되어 박사과정 수료학점에 포함하지 않는다.
라. 본 대학원 석사과정을 ...

결과 #2
   유사도: 0.559
   제목: 이수규정 – 서울대학교 데이터사이언스대학원
   URL: https://gsds.snu.ac.kr/academics/complete/
   내용 (앞 200자): 목의 경우 수료학점에 포함되지 않는다.
마. (삭제)
2. 박사과정
가. 교과목 구성 중 “공통” 분야에서 데이터사이언스 세미나(1학점) 세 번 이상과 이 외의 최소 1과목을 포함하여 6학점을, 선택과목에서 최소 18학점을 이수하여야 한다.
나. 논문연구과목의 경우 최소 6학점을 이수하여야 하며, 최대 12학점까지 수료학점에 포함된다.
다. 본 대학원 석사...

결과 #3
   유사도: 0.549
   제목: 이수규정 – 서울대학교 데이터사이언스대학원
   URL: https://gsds.snu.ac.kr/academics/complete/
   내용 (앞 200자): 미나(1학점) 세 번 이상과 이 외의 최소 1과목을 포함하여 6학점을, 선택과목에서 최소 18학점을 이수하여야 한다.
나. 논문연구과목의 경우 최소 6학점, 최대 12학점까지 이수할 수 있으며, 수료학점에 포함된다.
다. 박사과정 시 본 대학원 석사과정에서 수료 기준학점을 초과하여 취득한 학점(기초 분야 제외)에 대하여 12학점까지 박사과정 수료학점으로 

## Part D: RAG 파이프라인 완성

### RAG 파이프라인 구조

```
1. 사용자 질문
   ↓
2. 벡터 DB 검색 (retrieve) ← Part C에서 만듦
   ↓
3. 검색된 문서 + 질문을 LLM에 전달
   ↓
4. 근거 기반 답변 생성 (generate)
   ↓
5. 출처와 함께 답변 반환
```


In [14]:
CHAT_MODEL = "gpt-4o-mini"

SYSTEM_PROMPT = """
너는 '서울대학교 데이터사이언스대학원(GSDS) FAQ' 챗봇이다.

아래에 제공되는 '근거 문서 발췌' 내용만 사용해서 한국어로 답하라.

중요한 규칙:
1) 근거 문서에 없는 내용은 절대 추측하지 말고, "제공된 자료에서 확인되지 않습니다"라고 명확히 말하라.
2) 학기/연도/일정 등 변경 가능성이 있는 정보는 반드시 "최신 정보는 출처 링크를 확인해주세요"라고 안내하라.
3) 답변이 끝나면 사용한 출처 URL을 bullet point로 정리하라.
4) 가능한 한 근거 문서의 원문을 그대로 인용하여 정확성을 높여라.

답변 형식:
[질문에 대한 답변]

**출처:**
- [출처 URL 1]
- [출처 URL 2]
"""


### RAG 답변 함수

In [15]:
def answer_with_rag(question: str, k: int = 5):
    """
    RAG를 사용하여 질문에 답변합니다.

    Args:
        question: 사용자 질문
        k: 검색할 문서 수

    Returns:
        근거 기반 답변
    """
    # 1. 관련 문서 검색 (Retrieval)
    hits = search_documents(question, k=k)

    # 2. 컨텍스트 구성
    context_parts = []
    for i, hit in enumerate(hits, 1):
        context_parts.append(
            f"[문서 {i}] {hit['meta']['title']}\n"
            f"URL: {hit['meta']['url']}\n"
            f"내용:\n{hit['text']}"
        )

    context = "\n\n" + "="*70 + "\n\n".join(context_parts)

    # 3. 프롬프트 생성
    user_prompt = f"질문: {question}\n\n근거 문서 발췌:{context}"

    # 4. LLM 호출 (Generation)
    response = client.chat.completions.create(
        model=CHAT_MODEL,
        temperature=0,  # 일관된 답변을 위해 0으로 설정
        messages=[
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": user_prompt},
        ],
    )

    return response.choices[0].message.content


### RAG 챗봇 테스트

In [16]:
# 테스트 질문들
demo_questions = [
    "휴학 신청은 어디에서 어떻게 하나요?",
    "수강신청에서 장바구니는 어떻게 동작하나요?",
    "수강신청 변경/취소 기간은 언제인가요?",
    "석사 졸업에 필요한 학점은 몇 학점인가요?",
]

print("="*70)
print("🤖 GSDS FAQ RAG 챗봇 데모 🤖")
print("="*70)

for i, q in enumerate(demo_questions, 1):
    print(f"\n\n{'='*70}")
    print(f"질문 #{i}: {q}")
    print("="*70)

    answer = answer_with_rag(q, k=5)
    print(f"\n{answer}")

    if i < len(demo_questions):
        input("\n[Enter를 누르면 다음 질문 출력]")

🤖 GSDS FAQ RAG 챗봇 데모


질문 #1: 휴학 신청은 어디에서 어떻게 하나요?

휴학 신청은 서울대학교 포털을 통해 진행할 수 있습니다. 구체적인 방법은 다음과 같습니다:

1. 서울대학교 포털에 로그인합니다.
2. 학사정보 메뉴로 이동합니다.
3. 학적변동(신청) 메뉴에서 휴학·복학을 선택합니다.

휴학 신청 시 가사휴학 외의 경우에는 증빙서류를 첨부해야 하며, 창업휴학은 소속학과(부) 사무실에 사업계획서 등 직접 서류를 제출한 후 승인 요청을 해야 합니다. 또한, 휴학 신청 후에는 등록금 고지서를 출력할 수 없으니 유의하시기 바랍니다.

**출처:**
- [출처 URL 1](https://gsds.snu.ac.kr/2024%ED%95%99%EB%85%84%EB%8F%84-2%ED%95%99%EA%B8%B0-%ED%9C%B4%ED%95%99-%EB%B3%B5%ED%95%99%EA%B7%80-%EB%B3%B5%EC%A0%81%EC%9E%90%EC%9E%85%ED%95%99-%EB%B0%8F-%EB%93%B1%EB%A1%9D-%EC%9D%BC/)

[Enter를 누르면 다음 질문 출력]


질문 #2: 수강신청에서 장바구니는 어떻게 동작하나요?

장바구니는 수강신청 기간 동안 "실제 수강예정"인 과목을 신청하는 기능입니다. 장바구니에 담은 강좌가 수강정원 이내일 경우, 수강신청이 자동으로 확정됩니다. 그러나 수강정원 초과일 경우에는 장바구니 신청자들이 먼저 선착순으로 수강신청하게 됩니다.

**출처:**
- [2025학년도 2학기 수강신청 및 강의현황 안내](https://gsds.snu.ac.kr/2025%ED%95%99%EB%85%84%EB%8F%84-2%ED%95%99%EA%B8%B0-%EC%88%98%EA%B0%95%EC%8B%A0%EC%B2%AD-%EC%95%88%EB%82%B4/)

[Enter를 누르면 다음 질문 출력]


질문 #3: 수강신청 변경/취소 기간은 언제인가요?

수강신청 변경 및 취소 기간은 다음과 같습니다:

- 수강신청

### Hallucination 비교: RAG vs No-RAG

같은 질문을 RAG 없이와 RAG로 각각 답변해보고 차이를 확인합니다.

In [17]:
def answer_without_rag(question: str):
    """
    RAG 없이 LLM만 사용하여 답변합니다.
    (비교용)
    """
    response = client.chat.completions.create(
        model=CHAT_MODEL,
        temperature=0,
        messages=[
            {"role": "system", "content": "너는 친절한 한국어 도우미다."},
            {"role": "user", "content": question},
        ],
    )
    return response.choices[0].message.content

# 비교 테스트
comparison_q = "2023학번부터 석사 수료에 필요한 학점은 몇 학점인가요?"

print("="*70)
print("Hallucination 비교 테스트")
print("="*70)
print(f"\n질문: {comparison_q}\n")

print("\n" + "="*70)
print("(A) RAG 없이 (일반 LLM)")
print("="*70)
no_rag_answer = answer_without_rag(comparison_q)
print(no_rag_answer)

print("\n\n" + "="*70)
print("(B) RAG 사용 (근거 기반)")
print("="*70)
rag_answer = answer_with_rag(comparison_q, k=5)
print(rag_answer)

print("\n\n" + "="*70)
print("분석")
print("="*70)
print("""
- (A) No-RAG: LLM이 일반적인 지식으로 추측하여 답변
  → 부정확하거나 근거 없는 정보일 수 있음 (Hallucination 위험)

- (B) RAG: 실제 GSDS 문서를 검색하여 근거 기반 답변
  → 정확한 정보 + 출처 제공

이것이 RAG의 핵심 가치입니다!
""")

Hallucination 비교 테스트

질문: 2023학번부터 석사 수료에 필요한 학점은 몇 학점인가요?


(A) RAG 없이 (일반 LLM)
2023학번부터 석사 과정에서 수료에 필요한 학점은 대학 및 전공에 따라 다를 수 있습니다. 일반적으로 한국의 석사 과정은 24학점에서 36학점 정도가 요구되는 경우가 많습니다. 하지만 각 대학의 규정이나 전공의 특성에 따라 다를 수 있으므로, 정확한 학점 수는 해당 대학의 대학원 규정을 확인하는 것이 가장 좋습니다. 각 대학의 공식 웹사이트나 학사 안내서를 참고하시기 바랍니다.


(B) RAG 사용 (근거 기반)
2023학번부터 석사 수료에 필요한 학점은 "전공필수과목을 이수해야 하며, 데이터사이언스 세미나(1학점)를 세 번 이상 수강해야 하고, 선택과목으로 최소 1과목 이상을 이수할 것을 권장"합니다. 그러나 구체적인 수료에 필요한 총 학점 수는 제공된 자료에서 확인되지 않습니다.

최신 정보는 출처 링크를 확인해주세요.

**출처:**
- [https://gsds.snu.ac.kr/academics/complete/](https://gsds.snu.ac.kr/academics/complete/)


분석

- (A) No-RAG: LLM이 일반적인 지식으로 추측하여 답변
  → 부정확하거나 근거 없는 정보일 수 있음 (Hallucination 위험)

- (B) RAG: 실제 GSDS 문서를 검색하여 근거 기반 답변
  → 정확한 정보 + 출처 제공

이것이 RAG의 핵심 가치입니다!



### 추가가능한 기능들

시간이 남으면 다음 기능들을 추가해보세요:

1. **대화 루프**: 'quit' 입력 시까지 계속 질문 받기
   ```python
   while True:
       question = input("\n질문 (종료: quit): ")
       if question.lower() == 'quit':
           break
       # ... RAG 답변 ...
   ```

2. **검색 결과 수 조절**: 사용자가 k 값을 선택할 수 있게 하기
   ```python
   k = int(input("검색할 문서 수 (1-10): "))
   answer = answer_with_rag(question, k=k)
   ```

3. **검색된 문서도 함께 표시**: 어떤 문서를 참고했는지 보여주기
   ```python
   hits = search_documents(question, k=5)
   print("\n검색된 문서:")
   for i, hit in enumerate(hits, 1):
       print(f"{i}. {hit['meta']['title']} (유사도: {hit['similarity']:.3f})")
   ```

4. **질문 히스토리**: 이전 질문과 답변 저장하고 다시 보기

5. **비교 모드**: RAG vs No-RAG 답변을 나란히 보여주기

---

### RAG의 장단점

**장점:**
- 정확하고 신뢰할 수 있는 답변
- 출처 제공으로 검증 가능
- 최신 정보 반영 용이 (DB만 업데이트)
- 도메인 특화 지식 구축
- Hallucination 감소

**한계 및 개선 방향:**
- 검색 품질에 의존 → 하이브리드 검색 (키워드 + 벡터)
- 청킹 전략이 중요 → 다양한 청킹 방법 실험
- 관련 문서가 없으면? → Fallback 전략 필요
- 비용 (임베딩 API) → 오픈소스 임베딩 모델 고려

### 다음 단계

1. **고급 RAG 기법**
   - Hybrid Search (BM25 + Vector)
   - Query Expansion
   - Re-ranking
   - Multi-hop Reasoning

2. **프로덕션 배포**
   - FastAPI로 API 서버 구축
   - Streamlit/Gradio로 UI 구축
   - 캐싱 전략
   - 모니터링 및 평가

3. **평가 방법론**
   - 검색 품질 평가 (Precision, Recall)
   - 답변 품질 평가 (Faithfulness, Relevance)
   - A/B 테스트

### Reference

- [LangChain Documentation](https://python.langchain.com/docs/get_started/introduction)
- [ChromaDB Documentation](https://docs.trychroma.com/)
- [OpenAI Embeddings Guide](https://platform.openai.com/docs/guides/embeddings)
- [RAG Paper (Lewis et al., 2020)](https://arxiv.org/abs/2005.11401)

---
